# Recommendation System for Selecting Skills in Double Track High School Program Using AHP Method

This is an example implementation of the Analytic Hierarchy Process (AHP) decision-making algorithm to provide recommendations for skill areas that align with the preferences of students who will be participating in the Double Track High School Program.

### Skill Areas
There are 7 skill areas available for students to choose from. These skill areas include multimedia, electrotechnics, electrical, culinary, beauty care, fashion, and light vehicle/motorcycle arrangement.

### Criteria
There are 6 criteria used to assess the suitability of a skill area with a student's preferences. These criteria include the number of jobs, salary, gender, entrepreneurial opportunity, interest, and skill difficulty.

## AHP Steps

### 1. Define criteria, alternatives, and the pairwise comparison matrix

In [57]:
import pandas as pd
df = pd.read_csv("assets/dataset.csv")

criteria = df.columns
criteria = criteria.drop(labels="skill_field")
alternative = df.get(key="skill_field")

# determine pairwise comparison matrix
pairwise_matrix = [
    [1, 6, 2, 1, 1, 3],
    [1/6, 1, 3, 1, 1/6, 4],
    [1/2, 1/3, 1, 1, 1/5, 1/7],
    [1, 1, 1, 1, 1/3, 4],
    [1, 6, 5, 3, 1, 1],
    [1/3, 1/4, 7, 1/4, 1, 1]
]

length = len(criteria)


### 2. Normalize the pairwise comparison matrix

Normalize the pairwise comparison matrix by dividing each element in the matrix by the sum of the elements in the column.

In [58]:
calculated_sum = [0.0] * length
for row in range(length):
    for col in range(length):
        calculated_sum[row] += pairwise_matrix[col][row]
calculated_sum

[4.0, 14.583333333333332, 19.0, 7.25, 3.7, 13.142857142857142]

In [59]:
normalized_matrix = [[0.0] * length for _ in range(length)]
for row in range(length):
      for col in range(length):
        normalized_matrix[row][col] = (pairwise_matrix[row][col] / calculated_sum[col])
normalized_matrix

[[0.25,
  0.4114285714285715,
  0.10526315789473684,
  0.13793103448275862,
  0.27027027027027023,
  0.2282608695652174],
 [0.041666666666666664,
  0.06857142857142857,
  0.15789473684210525,
  0.13793103448275862,
  0.04504504504504504,
  0.30434782608695654],
 [0.125,
  0.022857142857142857,
  0.05263157894736842,
  0.13793103448275862,
  0.05405405405405406,
  0.010869565217391304],
 [0.25,
  0.06857142857142857,
  0.05263157894736842,
  0.13793103448275862,
  0.09009009009009009,
  0.30434782608695654],
 [0.25,
  0.4114285714285715,
  0.2631578947368421,
  0.41379310344827586,
  0.27027027027027023,
  0.07608695652173914],
 [0.08333333333333333,
  0.017142857142857144,
  0.3684210526315789,
  0.034482758620689655,
  0.27027027027027023,
  0.07608695652173914]]

### 3. Calculate the criteria weight vector

Calculate the criteria weight vector by averaging the elements in each row of the normalized pairwise comparison matrix.

In [60]:
criteria_weights = [0.0] * length
for x in range(length):
    criteria_weights[x] = sum(normalized_matrix[x]) / length
criteria_weights

[0.23385898394025909,
 0.12590945628249345,
 0.06722389592645255,
 0.15059532636310036,
 0.28078946606761646,
 0.14162287142007807]

### 4. Calculate the consistency index (CI) and consistency ratio (CR)

Calculate the consistency index (CI) by multiplying the pairwise comparison matrix with the criteria weight vector. Then, calculate the consistency ratio (CR) by dividing the CI by the random index (RI).

If the CR is less than 0.1, the pairwise comparison matrix is considered consistent.

In [61]:
temp = [[0.0] * length for _ in range(length)]
weighted_sum_value = [0.0] * length
ratio = [0.0] * length
for x in range(length):
    for y in range(length):
        temp[x][y] = pairwise_matrix[x][y] * criteria_weights[y]
        weighted_sum_value[x] += temp[x][y]
        
for x in range(length):
    ratio[x] += weighted_sum_value[x] / criteria_weights[x]

sum_temp = 0.0
for val in ratio:
    sum_temp += val
lambda_max = sum_temp / length    
ci = (lambda_max - length) / (length - 1)
random_index = [0.0, 0.0, 0.0, 0.58, 0.90, 1.12, 1.24, 1.32, 1.41, 1.45, 1.49] 
cr = ci / random_index[length]
if cr < 0.10:
    print(f"CR is OK\nCR = {cr}")
else:
    print(f"CR is BAD\nCR = {cr}")

CR is BAD
CR = 0.3113273970327574


In [62]:
# Show the criteria weights for each criteria
criteria_dict = {key: value for key, value in zip(criteria, criteria_weights)}
sorted_criteria =  dict(sorted(criteria_dict.items(), key=lambda x: x[1], reverse=True))
for criteria, weight in sorted_criteria.items():
    print(f"{criteria}: {weight}")

interest: 0.28078946606761646
number_of_jobs: 0.23385898394025909
entrepreneurial_opportunity: 0.15059532636310036
skill_dificulty: 0.14162287142007807
salary: 0.12590945628249345
gender: 0.06722389592645255


### 5. Ranke the alternatives

Create and normalized decision matrix by dividing each element in the decision matrix by the sum of the elements in the column. Then, calculate the weighted decision matrix by multiplying the normalized decision matrix with the criteria weight vector.

Finally, calculate the weighted sum of each alternative by summing the elements in each row of the weighted decision matrix. The alternative with the highest weighted sum is the recommended alternative.

In [63]:
decision_matrix = df.drop(labels="skill_field", axis=1).values
normalized_decision_matrix = []

for row in decision_matrix:
    row_sum = sum(row)
    normalized_row = [val / row_sum for val in row]
    normalized_decision_matrix.append(normalized_row)

weighted_scores = []
for row in normalized_decision_matrix:
    weighted_score = sum([val * weight for val, weight in zip(row, criteria_weights)])
    weighted_scores.append(weighted_score)

total_weighted_score = sum(weighted_scores)
overall_priorities = [score / total_weighted_score for score in weighted_scores]

ranked_alternatives = sorted(zip(alternative, overall_priorities), key=lambda x: x[1], reverse=True)

for key, value in ranked_alternatives:
    print(f"{key}: {value}")

multimedia: 0.1612649325020518
light_vehicle_arrangement: 0.15020063055049485
fashion: 0.1418922277959203
beauty_care: 0.13969825458473772
electrotechnics: 0.137840427046421
electrical: 0.13635168920823026
cullinary: 0.132751838312144
